<a href="https://colab.research.google.com/github/prashantiyaramareddy/AI-ML-Learnings/blob/master/ComputerVision/NN_overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import wandb
from wandb.integration.keras import WandbMetricsLogger

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
# Sweep config for selecting experiment
sweep_config = {
    'method': 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'batch_size':{'values': [8]},
        'learning_rate': {'values': [1e-3]},
        'hidden_nodes':{'values':[128]},
        'img_size':{'values':[16]},
        'epochs':{'values':[10]},
        'experiment':{'values':['dropout_only','batchnorm_only','full']}
    }
}

sweep_id = wandb.sweep(sweep_config, project="5_flowers_experiments")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: gy9otly7
Sweep URL: https://wandb.ai/prashantiyaramareddy7-nit-rourkela/5_flowers_experiments/sweeps/gy9otly7


In [3]:
def train():
  with wandb.init() as run:
    config = wandb.config

    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
      img_bytes = tf.io.read_file(filename)
      img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
      img = tf.image.resize(img, resize_dims)
      return img

    def parse_csvline(csv_line):
      record_default = ["",""]
      filename, label_string = tf.io.decode_csv(csv_line, record_defaults=record_default)
      img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
      label = tf.where(tf.equal(CLASS_NAMES, label_string))[0][0]
      return img, label

    # Datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    # Build Model
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)))

    if config.experiment == 'dropout_only':
      model.add(keras.layers.Dense(config.hidden_nodes, activation='relu'))
      model.add(keras.layers.Dropout(0.5))

    elif config.experiment == 'batchnorm_only':
      model.add(keras.layers.Dense(config.hidden_nodes, activation='relu'))
      model.add(keras.layers.BatchNormalization())
      model.add(keras.layers.Dropout(0.5))

    elif config.experiment == 'full':
      model.add(keras.layers.Dense(config.hidden_nodes,
                                   kernal_regularizer=keras.regularizers.l2(0.01),
                                   use_bias=False))
      model.add(keras.layers.BatchNormalization())
      model.add(keras.layers.Activation('relu'))
      model.add(keras.layers.Dropout(0.5))

    model.add(keras.layers.Dense(len(CLASS_NAMES), activation='softmax'))

    #Compile
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config.learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(
            from_logits=False
        ),
        metrics=["accuracy"]
    )

    callbacks = [WandbMetricsLogger(log_freq=5)]
    if config.experiment == 'full':
      callbacks.append(keras.callbacks.EarlyStopping
                       (monitor='val_loss', patience=3, restore_best_weights=True))

    # Train
    model.fit(
        train_dataset,
        epochs=config.epochs,
        validation_data=eval_dataset,
        callbacks=callbacks
    )

wandb.agent(sweep_id, train)








wandb: Agent Starting Run: poixrja9 with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: dropout_only
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: prashantiyaramareddy7 (prashantiyaramareddy7-nit-rourkela) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
    413/Unknown 189s 450ms/step - accuracy: 0.2191 - loss: 51.4782

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 209s 500ms/step - accuracy: 0.2192 - loss: 51.3909 - val_accuracy: 0.2378 - val_loss: 1.8246
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 162s 391ms/step - accuracy: 0.2429 - loss: 1.7060 - val_accuracy: 0.2378 - val_loss: 1.7485
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 340ms/step - accuracy: 0.2460 - loss: 1.6216 - val_accuracy: 0.2378 - val_loss: 1.7078
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 111s 269ms/step - accuracy: 0.2402 - loss: 1.6196 - val_accuracy: 0.2324 - val_loss: 1.7109
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 106s 256ms/step - accuracy: 0.2436 - loss: 1.6011 - val_accuracy: 0.2351 - val_loss: 1.7280
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 102s 247ms/step - accuracy: 0.2430 - loss: 1.6275 - val_accuracy: 0.2351 - val_loss: 1.7194
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 101s 245ms/step - accuracy: 0.2446 - loss: 1.5912 - val_accuracy: 0.2351 - val_loss: 1.7189
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 101s 246ms/step - accuracy: 0.2444 - loss: 1.5

...

Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 99s 232ms/step - accuracy: 0.3146 - loss: 1.7198 - val_accuracy: 0.4270 - val_loss: 1.5801
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 95s 231ms/step - accuracy: 0.4327 - loss: 1.3539 - val_accuracy: 0.4297 - val_loss: 1.7972
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 94s 228ms/step - accuracy: 0.4447 - loss: 1.3211 - val_accuracy: 0.4297 - val_loss: 1.8266
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 90s 219ms/step - accuracy: 0.4458 - loss: 1.2964 - val_accuracy: 0.4459 - val_loss: 1.8379
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 150s 237ms/step - accuracy: 0.4736 - loss: 1.2779 - val_accuracy: 0.4703 - val_loss: 1.8705
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 133s 216ms/step - accuracy: 0.4641 - loss: 1.2628 - val_accuracy: 0.4649 - val_loss: 1.9351
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 93s 225ms/step - accuracy: 0.4687 - loss: 1.2591 - val_accuracy: 0.4730 - val_loss: 2.1306
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 91s 221ms/step - accuracy: 0.4495 - loss:

batch/accuracy,▁▁▂▃▄▇▆▇▆▆▇▇▇▇▇▇▇████▇██▇▇▇███▇▇▇▇▇▇▇▇▇▇
batch/batch_step,▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▆▅▅▅▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▁▁▂▂▁▂▂▂▂▂▂▂
epoch/accuracy,▁▅▆▆█▇▇▇▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▂▁▁▁▁▁▁
epoch/val_accuracy,▁▁▁▄█▇█▂▃▃
epoch/val_loss,▁▄▄▄▅▆████
batch/accuracy,0.46229


wandb: Agent Starting Run: vqgxbt1y with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: full
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001


Traceback (most recent call last):
  File "/tmp/ipython-input-2017578586.py", line 52, in train
    model.add(keras.layers.Dense(config.hidden_nodes,
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py", line 93, in __init__
    super().__init__(activity_regularizer=activity_regularizer, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py", line 291, in __init__
    raise ValueError(
ValueError: Unrecognized keyword arguments passed to Dense: {'kernal_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x7ec2905278c0>}


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/wandb/agents/pyagent.py", line 296, in _run_job
    self._function()
  File "/tmp/ipython-input-2017578586.py", line 52, in train
    model.add(keras.layers.Dense(config.hidden_nodes,
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py", line 93, in __init__
    super().__init__(activity_regularizer=activity_regularizer, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py", line 291, in __init__
    raise ValueError(
ValueError: Unrecognized keyword arguments passed to Dense: {'kernal_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x7ec2905278c0>}

wandb: ERROR Run vqgxbt1y errored: Unrecognized keyword arguments passed to Dense: {'kernal_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x7ec2905278c0>}
wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Ex